In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import joblib

In [2]:
# Read in the data. Label is the numeric label (0-9). Other columns in X data represent the pixel intensity (0-255) of the image
# at the designated pixel. The MNIST data is already quite clean and well pre-processed, so we can just feed the data into
# our model.

X_train = pd.read_csv("mnist_train.csv")
y_train = X_train.label
X_train.drop('label', axis = 1, inplace = True)

X_test = pd.read_csv("mnist_test.csv")
y_test = pd.DataFrame(X_test.label)
X_test.drop('label', axis = 1, inplace = True)

X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [3]:
from scipy.ndimage.interpolation import shift

# Function that shifts the original images by dy in the y direction and dx in the x direction (using image coordinates)
# i.e. the origin is in the top left corner of the image.
def shift_digit(image, dx, dy):
    image = image.reshape(28,28)
    shift_image = np.array(shift(image, [dy, dx], cval = 0, mode = "constant"))
    return shift_image.reshape([-1])


# Convert to a list, to make it less of a hassle to append to the data set
X_train_expanded = X_train.tolist()
y_train_expanded = y_train.tolist()

# Augment the data, by shifting each image four pixels to the right, four pixels to the left, four pixels down,
# then four pixels up. This will lead to better generalization and accuracy.
for dx, dy in ((4,0), (-4, 0), (0, 4), (0, -4)):
    for ix in range (0, len(X_train)):
        shifted_image = shift_digit(X_train[ix], dx, dy)
        X_train_expanded.append(shifted_image.tolist())
        y_train_expanded.append(y_train[ix])

In [4]:
# # Testing shift_digit function
# original = X_train_expanded[1]
# shifted = X_train_expanded[60001]

# plt.imshow(original.reshape(28,28), cmap = "Greys")
# plt.show()
# plt.imshow(shifted.reshape(28,28), cmap = "Greys")
# plt.show()

In [11]:
from sklearn.ensemble import RandomForestClassifier

image_clf = RandomForestClassifier(n_estimators = 250, n_jobs = 12)
image_clf.fit(X_train_expanded, y_train_expanded)

RandomForestClassifier(n_estimators=250, n_jobs=12)

In [12]:
y_test_pred = image_clf.predict(X_test)
accuracy = image_clf.score(X_test, y_test)
print (accuracy)

0.9701


In [14]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

y_train_pred = cross_val_predict(image_clf, X_train, y_train, cv = 3, n_jobs = 12)
conf_matrix = confusion_matrix(y_train, y_train_pred)
print (conf_matrix)

[[5842    1    8    0    4    6   19    1   38    4]
 [   1 6636   40   11   11    3    7   11   14    8]
 [  32   10 5751   31   32    3   18   37   36    8]
 [   9    5   81 5814    4   66    9   50   65   28]
 [  10   11   10    1 5660    0   29   12   17   92]
 [  22    7   10   60    8 5205   45    5   31   28]
 [  25   12    3    0    8   37 5815    0   18    0]
 [   7   24   55    9   33    1    0 6046   14   76]
 [  10   31   33   45   26   38   26    5 5573   64]
 [  22    9   11   73   63   19    4   49   45 5654]]


## Random Forest Classifier Model Evaluation:

Model_v3 utilizes a RandomForestClassifier with 250 decision trees. It yields a marginally better accuracy score on the test data (97% as opposed to model_v2's 96.72%). However, it is MUCH faster since it utilizes Decision trees instead of the Nearest Neighbors algorithm. 

The Confusion Matrix for this data set (columns from 0-9, representing the classified digit):


    [[5842    1    8    0    4    6   19    1   38    4]
     [   1 6636   40   11   11    3    7   11   14    8]
     [  32   10 5751   31   32    3   18   37   36    8]
     [   9    5   81 5814    4   66    9   50   65   28]
     [  10   11   10    1 5660    0   29   12   17   92]
     [  22    7   10   60    8 5205   45    5   31   28]
     [  25   12    3    0    8   37 5815    0   18    0]
     [   7   24   55    9   33    1    0 6046   14   76]
     [  10   31   33   45   26   38   26    5 5573   64]
     [  22    9   11   73   63   19    4   49   45 5654]]

As evidenced by the confusion matrix, model performance on 8's, 6's, and 9's are the worst. 6's often get classified as 5's, 8's often get classified as 3's, and 9's often get classified as 4's. These all make sense since these numbers are very structurally similar, so this type of error will always exist to some degree (and sometimes even humans can't tell the difference).

In [15]:
# Save model to file called "KNN_MNIST_ImageClassifier_v3"
joblib.dump(image_clf, "KNN_MNIST_ImageClassifier_v3")


['KNN_MNIST_ImageClassifier_v3']